# Overview

This notebook allows to use the `AMReX` library interactively in the Jupyter notebook.

This is meant to help WarpX developers, by providing a way to quickly test/prototype AMReX-based code.

# Setup

- Compile `AMReX` as a library as a dynamic library: from the `amrex` root source folder:
```
mkdir builddir
cd builddir
cmake .. -DBUILD_SHARED_LIBS=ON -DENABLE_MPI=OFF
make install
```
(For MacOS, see [this page](https://amrex-codes.github.io/amrex/docs_html/BuildingAMReX.html#cmake-and-macos))

This will create the library files in `tmp_install_dir/include` and `tmp_install_dir/lib`.

- Create a new `conda` environment and install `xeus-cling`
For Linux:
```
conda create -n xeus
source activate xeus
conda install -c conda-forge xeus-cling jupyter
```
For MacOS:
```
conda create -n xeus
source activate xeus
conda install -c QuantStack -c conda-forge xeus-cling jupyter
```

- **Optional:** in order to link the documentation:
In the source folder of amrex, open the file `Docs/Doxygen/doxygen.conf` and change the line `GENERATE_TAGFILE       = ` to `GENERATE_TAGFILE       = amrex.tag.xml`, then run `doxygen doxygen.conf`. This creates a file `amrex.tag`. Copy this file to `$HOME/miniconda3/envs/xeus/share/xcpp/tagfiles/`. Then create a file `amrex.json` in `$HOME/miniconda3/envs/xeus/etc/tags.d/`, and enter the following text:
```
{
    "url": "https://amrex-codes.github.io/amrex/doxygen/",
    "tagfile": "amrex.tag"
}
```

- Copy this notebook outside of the source folder of WarpX (so that it is not tracked by `git`), and open it with `source activate xeus ; jupyter notebook`.

- Change the paths of the cells below to the path of your amrex installation, and execute them

In [ ]:
#pragma cling add_include_path("/home/rlehe/Documents/codes/warpx_directory/amrex/install_nompi_dynamic/include")
#pragma cling add_library_path("/home/rlehe/Documents/codes/warpx_directory/amrex/install_nompi_dynamic/lib")
#pragma cling load("libamrex.so")

In [ ]:
#define AMREX_SPACEDIM 3
#include <AMReX.H>

In [ ]:
MPI_Comm dummy_mpi_comm;
amrex::Initialize(dummy_mpi_comm);

# Example: typical MultiFab creation / looping

- Create a box that represents the full domain, and a split it into a BoxArray

In [ ]:
#include <AMReX_Print.H>
#include <AMReX_IntVect.H>
#include <AMReX_Box.H>
#include <AMReX_BoxArray.H>
using namespace amrex;

In [ ]:
Box domain(IntVect{0,0,0}, IntVect{127,127,127});
BoxArray ba(domain);  // Make a new BoxArray of size 128^3
Print() << ba;

In [ ]:
ba.maxSize(64);       // Chop into boxes of size 64^3
Print() << ba;

In [ ]:
?ba     // ba is a BoxArray ; get the doc for this

In [ ]:
?ba[0]

- Create a distribution mapping over the created BoxArray

In [ ]:
#include <AMReX_DistributionMapping.H>

In [ ]:
DistributionMapping dm{ba}; // Create distribution mapping from BoxArray

In [ ]:
// Here, the mapping is trivial since we are using a single MPI proc
Print() << "Number of boxes: " << dm.size() << std::endl;
for (int i=0; i<dm.size(); i++){
    Print() << "Box number " << i << " is owned by MPI proc " << dm[i] <<std::endl;
}

- Create a corresponding MultiFab

In [ ]:
#include <AMReX_MultiFab.H>

In [ ]:
int ncomp = 1;
int ngrow = 0;
MultiFab mf(ba, dm, ncomp, ngrow);

In [ ]:
// Right now, the MultiFab allocated with non-sensical data, as it has not been initialize
Print() << mf[0];

In [ ]:
mf.setVal(0);
Print() << mf[0];